In [1]:
import os
from supabase import create_client, Client
import requests
from dotenv import load_dotenv

In [2]:
# Loading environment variables
load_dotenv()
KOBO_API_KEY = os.getenv("KOBO_API_KEY")
KOBO_FORM_ID = os.getenv("KOBO_FORM_ID")
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_API_KEY = os.getenv("SUPABASE_API_KEY")

In [4]:
# Initialize Supabase client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_API_KEY)

In [ ]:
# Fetch data from Kobo
def fetch_kobo_data():
    headers = {"Authorization": f"Token {KOBO_API_KEY}"} # should this be in dictionary format or just normal
    url = f"https://kf.kobotoolbox.org/api/v2/assets/{KOBO_FORM_ID}/data"
    response = requests.get(url, headers=headers)
    return response.json()["results"]


In [7]:
def fetch_kobo_data_debug():
    print(f"API Key: {KOBO_API_KEY}")
    print(f"Form ID: {KOBO_FORM_ID}")
    
    headers = {"Authorization": f"Token {KOBO_API_KEY}"}
    url = f"https://kf.kobotoolbox.org/api/v2/assets/{KOBO_FORM_ID}/data"
    
    print(f"Request URL: {url}")
    
    response = requests.get(url, headers=headers)
    
    print(f"Status Code: {response.status_code}")
    print(f"Response Headers: {response.headers}")
    print(f"Raw Response: {response.text[:500]}...") 
    
    return response.json() #should this be parsed using the .json() or .json() loads commands?

debug_response = fetch_kobo_data_debug()

API Key: 04ce34d1b625e50a59c2411bfe865e41e5be49f5
Form ID: amMeFyvFFey55X5rF6Sm5g
Request URL: https://kf.kobotoolbox.org/api/v2/assets/amMeFyvFFey55X5rF6Sm5g/data
Status Code: 200
Response Headers: {'Date': 'Thu, 24 Jul 2025 06:59:16 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'nginx', 'Vary': 'Accept, origin, Cookie, Accept-Language', 'Allow': 'GET, HEAD, OPTIONS', 'Content-Security-Policy': "img-src 'self' https://kc.kobotoolbox.org https://ee.kobotoolbox.org data: https://*.openstreetmap.org https://*.openstreetmap.fr https://*.opentopomap.org https://*.arcgisonline.com https://*.google-analytics.com https://*.googletagmanager.com; style-src 'self' https://kc.kobotoolbox.org https://ee.kobotoolbox.org 'unsafe-inline'; connect-src 'self' https://kc.kobotoolbox.org https://ee.kobotoolbox.org https://*.googletagmanager.com https://*.google-analytics.com https://*.analytics.google.com https://*.googletagmanager

JSONDecodeError: Expecting value: line 5 column 1 (char 4)

In [17]:
def process_submissions():
    submissions = fetch_kobo_data()
    for submission in submissions:
        data = {
            "kobo_submission_id": submission["_id"],  # Required field (no .get())
            "dusun": submission.get("dusun", ""),      # Optional field
            # ... other fields
        }
        supabase.table("survey_responses").insert(data).execute()